<img style="float: right;" src="imgs/unifor.jpg" width="150px">

# Política no Twitter

Usamos redes sociais para estabelecer contatos, exposição profissional e expressar opiniões. Redes como [Facebook](https://www.facebook.com/) e [Twitter](https://twitter.com/) recebem diariamente milhões de postagens e comentários de seus usuários. Considerando esse cenário, _seria possível observar padrões de comportamento em mensagens postadas no Twitter?_

Este Notebook analisa dados da conta de dois políticos brasileiros, [Lula](https://twitter.com/LulaOficial) e [Jair Bolsonaro](https://twitter.com/jairbolsonaro) na tentativa de identificar esses padrões. 

Trabalho como resultado de Projeto da disciplina de **Introdução a Ciência de Dados**.

## Ambiente
Utilizamos [Python](https://www.python.org/) no ambiente [Jupyter](https://jupyter.org/) para desenvolver os Notebooks.

Para montar o ambiente de execução você precisará de uma versão do [Conda](https://sandbox.anaconda.com/) e opcionalmente o [Git](https://git-scm.com/). Você encontrará instrução de instalação desses produtos em seus próprios sites.

```bash
git clone https://github.com/michelav/projeto-twitter.git
cd projeto-twitter
source path_to_conda/activate
conda env create -f environment.yml
conda activate ptwitter
jupyter-lab
```

## Dados

O _dataset_ contém todos os tweets publicados pelos dois políticos até o fim de 2019. Esses tweets estão divididos em dois (um para cada político) arquivos texto no formato ```json```. Os arquivos foram extraídos por meio da API Web da plataforma e seguem o formato definido no [_hotsite_](https://developer.twitter.com/en/docs/tweets/data-dictionary/overview/intro-to-tweet-json) da API.

## Análise Exploratória

Para a análise exploratória utilizaremos a biblioteca Pandas. Ele permite que organizemos os dados em _frames_ que permitem consulta, agregação e processamento massivo de dados.

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
import os
import pandas as pd
import numpy as np
import datetime as dt
import json
from pandas.api.types import CategoricalDtype


dfs = []
profiles = []
to_be_dropped = ['id', 'display_text_range', 'source', 
                 'in_reply_to_status_id', 'in_reply_to_status_id_str', 
                 'in_reply_to_user_id', 'in_reply_to_user_id_str', 
                 'in_reply_to_screen_name', 'user', 'geo', 'coordinates', 'place',
                 'contributors', 'is_quote_status', 'lang', 'extended_entities',
                 'quoted_status_id', 'quoted_status_id_str', 'quoted_status_permalink']


# object_hook para processar as strings do json como datetime
def process_tweets(dct):
    # Chaves e sub-chaves usadas para decompor as informacoes de hashtags e mencoes 
    meta = {'hashtags': 'text', 'user_mentions': 'screen_name'}
    if 'created_at' in dct:
        try:
            dct['created_at'] = dt.datetime.strptime(dct['created_at'],'%a %b %d %H:%M:%S %z %Y').replace(hour=0, minute=0, second=0, microsecond=0)
        except ValueError:
            dct['created_at'] = np.nan
    
    for k, sk in meta.items():
        # Procura pelas chaves de Meta no dicionario que represanta o Json
        if k in dct:
            # Itera pela lista de dicionarios em cada chave encontrada(ver estrutura do json) e agrega as sub-chaves em uma lista
            items = dct[k]
            dct[k] = [ivalue for item in items for ikey, ivalue in item.items() if ikey == sk] if items else np.nan
    return dct

with os.scandir('dados') as lsit:
    fit = (f for f in lsit if f.is_file())
    for f in fit:
        with open(f.path, mode='r') as fp:
            profile = f.name.split('.')[0]
            profiles.append(profile)
            tweets = json.load(fp, object_hook=process_tweets)
            df = pd.DataFrame(tweets)
            df.drop(columns=to_be_dropped, inplace=True)
            df['tweet_len'] = df['full_text'].str.len()
            df['profile'] = profile
            df = df.join(pd.json_normalize(df['entities']).drop(columns=['symbols', 'urls', 'media']))
            dfs.append(df)

tweets_df = pd.concat(dfs)
cat_type = CategoricalDtype(categories=profiles , ordered=True)
tweets_df['profile'] = tweets_df['profile'].astype(cat_type)

In [ ]:
tweets_df.head(5)

+ Criando um índice baseado no perfil do usuário, data de criação e identificação do tuíte.

In [ ]:
# mindex = pd.MultiIndex.from_frame(tweets_df[['profile', 'created_at', 'id_str']])
tweets_df.set_index(['profile', 'created_at', 'id_str'], inplace=True)
tweets_df.sort_index(inplace=True)
mindex = tweets_df.index

+ Utilizando a biblioteca spacy para processar o texto dos tuítes

In [ ]:
from collections import Counter
import pt_core_news_sm
import spacy
from spacy.tokens import Token
from spacymoji import Emoji

# Regras para desconsiderar tags e mentions na contagem de palavras
is_hashtag_getter = lambda token: len(token.text) > 1 and token.text.startswith('#')
is_mention_getter = lambda token: len(token.text) > 1 and token.text.startswith('@')
is_currency_getter = lambda token: token.text.lower() == 'r$'
is_abrev_getter = lambda token: len(token.text) <= 2 and token.text.lower() in ['c/', 'p/', 'q']

Token.set_extension("is_hashtag", getter=is_hashtag_getter, force=True)
Token.set_extension("is_mention", getter=is_mention_getter, force=True)
Token.set_extension("is_currency", getter=is_currency_getter, force=True)
Token.set_extension("is_abrev", getter=is_abrev_getter, force=True)

# Novas stop words para PT 
custom_stop_words = ['a', 'e', 'o', 'n', 'd', 'A', 'E', 'O', 'N', 'D']

nlp = pt_core_news_sm.load()

# Configurando para remover emojis
emoji = Emoji(nlp, merge_spans=False)
nlp.add_pipe(emoji, first=True)

# Configurando o modelo com as novas stop words
for sw in custom_stop_words:
    nlp.vocab[sw].is_stop = True

# Configurando prefixos para n separar tags e @
prefixes = list(nlp.Defaults.prefixes)
# prefixes
prefixes.remove("#")
# prefixes.append('R\\$')
prefix_regex = spacy.util.compile_prefix_regex(prefixes)
nlp.tokenizer.prefix_search = prefix_regex.search

docs = list(nlp.pipe(tweets_df['full_text'].str.replace(r'\n', '').to_numpy(),
                     disable=["tagger", "parser", "ner", "textcat"]))

In [ ]:
def process_doc(doc):
    return {'vocab_len': len(doc), 'palavras': [token.text.lower() for token in doc if  not token.is_stop 
         and not token.is_punct and not token.like_url and not token._.is_hashtag 
         and not token._.is_mention and not token._.is_currency and not token._.is_abrev
         and not token._.is_emoji]}

tweets_df = tweets_df.join(pd.DataFrame(list(map(process_doc, docs)), index=mindex, columns=['vocab_len', 'palavras']))
tweets_df

### Quantidade de tweets por mês

In [ ]:
# tweets_df.groupby(['profile', 'created_at'])['full_text'].count()
# years = tweets_df.index.levels[1].year
# months = tweets_df.index.levels[1].month
di = pd.Grouper(freq='M', level='created_at')
pi = pd.Grouper(level='profile')

tweets_df.groupby([pi, di])['full_text'].count().head(50)

# dir(g)
# g.groups
# g.get_group('jairbolsonaro')

In [ ]:
s = tweets_df.loc['jairbolsonaro', 'palavras'].explode()
s.value_counts().head(15)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

fig, ax = plt.subplots()
labels, data = zip(*words_freq.most_common(10))

x = np.arange(len(labels))  # the label locations

# Add some text for labels, title and custom x-axis tick labels, etc.
ax.set_ylabel('Frequência')
ax.set_title('Palavras mais utilizadas pelo Lula')

plt.bar(x, list(data))
plt.xticks(x, labels, rotation=80)
fig.tight_layout()
plt.show()

In [ ]:
a = [1, 2]

b = 'Cheio' if a else 'Vazio'
b

In [ ]:
df = pd.DataFrame(np.random.randn(3, 8), index=['A', 'B', 'C'], columns=index)

In [ ]:
print(('olá', 1))